In [1]:
from osgeo import gdal
import os
import glob
import math
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp

In [2]:
#获取影像的左上角和右下角坐标
def GetExtent(in_fn):
    ds=gdal.Open(in_fn)
    geotrans=list(ds.GetGeoTransform())
    xsize=ds.RasterXSize 
    ysize=ds.RasterYSize
    min_x=geotrans[0]
    max_y=geotrans[3]
    max_x=geotrans[0]+xsize*geotrans[1]
    min_y=geotrans[3]+ysize*geotrans[5]
    ds=None
    return min_x,max_y,max_x,min_y

In [3]:
path=r"C:\\2012_mask"
os.chdir(path)
#如果存在同名影像则先删除
# if os.path.exists('LaiParamOneCycle_2018.tif'):
#     os.remove('LaiParamOneCycle_2018.tif')
    
in_files=glob.glob("*.tif")#得到该目录下所有的影像名
in_fn=in_files[0]
#获取待镶嵌栅格的最大最小的坐标值
min_x,max_y,max_x,min_y=GetExtent(in_fn)
for in_fn in in_files[1:]:
    minx,maxy,maxx,miny=GetExtent(in_fn)
    min_x=min(min_x,minx)
    min_y=min(min_y,miny)
    max_x=max(max_x,maxx)
    max_y=max(max_y,maxy)
print(min_x,max_y,max_x,min_y)

-180.00441663186973 90.00220831593487 180.0044166318697 -90.00220831593487


In [4]:
#计算镶嵌后影像的行列号
in_ds=gdal.Open(in_files[0])
geotrans=list(in_ds.GetGeoTransform())
width=geotrans[1]
height=geotrans[5]
columns=math.ceil((max_x-min_x)/width)
rows=math.ceil((max_y-min_y)/(-height))
band_type=in_ds.GetRasterBand(1)
in_band=in_ds.RasterCount

In [5]:
name="D:\LAI_fitting_param\download_lai_param_twoCycle\2012mask.tif"
out_ds=gdal.GetDriverByName('GTiff').Create(name,columns,rows,in_band,band_type.DataType)
out_ds.SetProjection(in_ds.GetProjection())
geotrans[0]=min_x
geotrans[3]=max_y
out_ds.SetGeoTransform(geotrans)

#定义仿射逆变换
inv_geotrans=gdal.InvGeoTransform(geotrans)

In [6]:
def GetData(in_filename):
    in_ds=gdal.Open(in_filename)
    in_gt=in_ds.GetGeoTransform()
    #仿射逆变换
    offset=gdal.ApplyGeoTransform(inv_geotrans,in_gt[0],in_gt[3])
    x,y=map(int,offset)
    for j in range(in_band):
        data=in_ds.GetRasterBand(j+1).ReadAsArray()
        out_ds.GetRasterBand(j+1).WriteArray(data,x,y)
    return True

In [7]:
if __name__ == '__main__':
    star=time.time()
    pool = ProcessPoolExecutor(max_workers=36)
    list(pool.map(GetData,in_files))
    pool.shutdown(wait=True)
    
    #     for j in range(in_band):
#         out_ds.GetRasterBand(j+1).WriteArray(data[j,:,:],x,y)
#     pool = mp.Pool(processes=18)
#     res2=[pool.apply_async(GetData,(i,)) for i in in_files]
    print("total time:",time.time()-star)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.